# 対話してみる

In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_ollama import ChatOllama, OllamaEmbeddings

import settings as sets

## 定数定義

In [2]:
NEO4J_URI: str = sets.NEO4J_URI
NEO4J_USER: str = sets.NEO4J_USER
NEO4J_PASSWORD: str = sets.NEO4J_PASSWORD

MODEL_NAME = 'elyza:8b'

## 単純なベクトルサーチをやってみる

In [3]:
user_query = 'SQLインジェクションについて教えて'

In [4]:
%%time

# グラフDB（リレーション含まない、すなわちただのベクトルDBと同等）
exist_db = Neo4jVector.from_existing_graph(
    embedding=OllamaEmbeddings(model=MODEL_NAME),
    node_label='Chunk',
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    text_node_properties=['text'],
    embedding_node_property='embedding'
)

CPU times: user 793 ms, sys: 219 ms, total: 1.01 s
Wall time: 5.98 s


In [5]:
exist_db.similarity_search_with_relevance_scores(user_query)

[(Document(metadata={'reference_text': '3.4失敗例(不適切なセッション管理)'}, page_content='\ntext: 「093a2031a79cb4904b1466ee7ad5faaa3afe7b787db66712f407326b213cc2a4」等です。このプログラムではセッションIDにハッシュ値を使っているため、一見、安全そうに見えるかもしれません。しかし、セッションIDの生成方法が第三者に知られた場合光には、第三者がセッションIDを推測できる余地があります。\n罠のウェブサイトへの誘導等によって、攻撃者は利用者の接続元IPアドレスを入手できますぎ。一方、ポート番号は攻撃者が知り得えない情報です。しかし、接続元ポート番号の範囲は1024から65535であり、利用者のネットワーク環境によってはこの範囲が2万通り程度に限定される場合があります。\nこのプログラムがオープンソースで開発されている場合、またはソースコードが漏えいしてしまった場合等を想定できます。5ウェブサイトに到達するまでのネットワーク経路によっては、特定のipアドレスとならない場合があります\n。\nな'),
  0.7720184326171875),
 (Document(metadata={'reference_text': '2.7携帯ウェブ向けのサイトにおける注意点'}, page_content='\ntext: 4ここで使用するセッションIDは、第三者に推測されない必要があります。詳しくは18ページの根本的解決4-()を参照してください。60\nな'),
  0.7718048095703125),
 (Document(metadata={'reference_text': '1.5クロスサイト・スクリプティング'}, page_content='\ntext: 1.5クロスサイト・スクリプティング2)に該当するウェブアプリケーションの例には、自由度の高い掲示板やブログ等が挙げられます。たとえば、利用者が入力文字の色やサイズを指定できる機能等を実装するために、HTMLテキストの入力を許可する場合があるかもしれません。\n3)は、1、2)の両者のウェブアプリケーションに共通して必要な対策です。\n1.5.1HTMLテキストの入力を許可

## グラフDBを活用した回答生成をやってみる

In [6]:
# グラフDB（リレーション含む）
graph_db = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD
)

In [7]:
graph_chain = GraphCypherQAChain.from_llm(
    ChatOllama(model=MODEL_NAME, temperature=0.0),
    graph=graph_db,
    verbose=True,
    allow_dangerous_requests=True
)

In [8]:
graph_chain.invoke(user_query)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Topic)-[:reference]->(c:Chunk) RETURN c.text
Full Context:
[{'c.text': 'ウェブアプリケーションのセキュリティ実装とウェブサイトの安全性向上のための取り組み\nIDA独立行政法人情報処理推進機構セキュリティセンター\n2012年12月\n本書は、以下のURLからダウンロードできます。「安全なウェブサイトの作り方」\nおわりに..…参考資料..eoメールヘッダ・インジェクションの例'}, {'c.text': "プレースホルダに実際の値を割り当てる処理をバインドと呼びます。バインドの方式には、プレースホルダのままSQL文をコンパイルしておき、データベースエンジン側で値を割り当てる方式(静的プレースホルダ)と、アプリケーション側のデータベース接続ライブラリ内で値をエスケーブプ処理してプレースホルダにはめ込む方式(動的プレースホルダ)があります。静的プレースホルダは、SQLのISO/JIS規格では、準備された文(PreparedStatement)と呼ばれます。どちらを用いてもSQLインジェクション脆弱性を解消できますが、原理的にSQLインジェクション脆弱性の可能性がなくなるという点で、静的プレースホルダの方が優ります。詳しくは本書別冊の「安全なSQLの呼び出し方」のプレースホルダの項(3.2節)を参照してください。\nごも\nSQL文の組み立てを文字列連結により行う場合は、エスケーブプ処理等を行うデータベースエンジンのAPIlを用いて、SQL文のリテラルを正しく構成する。\nSQL文の組み立てを文字列連結により行う場合は、SQL文中で可変となる値をリテラル(定数)の形で埋め込みます。値を文字列型として埋め込む場合は、値をシングルクォートで囲んで記述しますが、その際に文字列リテラル内で特別な意味を持つ記号文字をエスケープ処理します(たとえば、「'」つつ「'「」、「\\」つ「」等)。値を数値型として埋め込む場合は、数値リテラルであることを確実にする処"}, {'c.text': '$最新情報は、下記URLを参照してください。\n脆弱性関連情報に関する届出状況:\n

{'query': 'SQLインジェクションについて教えて',
 'result': "SQLインジェクションとは、Webアプリケーションの入力フォームなどを通じて、悪意のあるユーザーがデータベースに不正な命令を送信し、データを改ざんや漏えいさせる攻撃手法です。\n\n具体的には、Webアプリケーションがデータベースとの通信で使用するSQL文に、特別な文字列を埋め込むことで、意図的にデータベースの動作を変更したり、情報を取得したりすることができます。\n\n例えば、入力フォームに「' OR 1=1 --」などの特殊な文字列を入力し、Webアプリケーション安全等/tech/sqlinjection.html"}

In [10]:
user_query2 = 'WAFの効果について例を教えて'
graph_chain.invoke(user_query2)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Topic {topic:"WAF"})-[:reference]->(c:Chunk) RETURN c.text
Full Context:
[]

> Finished chain.


{'query': 'WAFの効果について例を教えて',
 'result': 'WAFは、Web Application Firewallの略で、Webアプリケーション層での攻撃を防ぐためのセキュリティ対策です。'}